In [1]:
import geopandas as gpd

import numpy as np
import pandas as pd
from pandas import IndexSlice as idx
import tensorflow as tf
import sys
import os
import glob

from functools import partial
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt




code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/perturbations/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/diff_bpr'
sys.path.append(code_dir)
from top_k import top_k_idx
#from make_datasets import make_data
from bpr_model import PerturbedBPRLinearModel, PerturbedBPRMLPModel
from make_datasets import make_data


code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)
from zinf_gp.metrics import normcdf, fixed_top_X



from perturbations import perturbed
from bpr import bpr_variable_k_no_ties



2023-07-06 05:12:39.974902: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-06 05:12:40.161244: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-06 05:12:40.313686: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-06 05:12:40.314593: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 05:12:45.415796: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
data_path='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/clean_annual_tract/'


In [50]:
best_model = 'n0.3_samp50_lr0.01_hs50_10_lb5'
noise = 0.3
perturbation_samples = 50
hidden_sizes = [50, 10]
lookback_years=5
learning_rate=0.01
add_spacetime=True
add_svi=True

In [51]:
epochs = 3000
seed = 360
time_window = lookback_years
timesteps_per_year = 1
first_train_eval_year = 2013
last_train_eval_year = 2017
batch_dim_size = (last_train_eval_year - first_train_eval_year + 1)*timesteps_per_year
validation_year = 2018
first_test_year = 2019
last_test_year = 2020


tf.random.set_seed(seed)


timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile', 'year',
              'neighbor_t', 'deaths']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['deaths']

if add_spacetime:
    features_only += ['lat', 'lon', timestep_col]
if add_svi:
    features_only += ['theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile']


data_gdf = gpd.read_file(data_path)

multiindexed_gdf = data_gdf.set_index(['geoid', 'timestep'])
multiindexed_gdf['timestep'] = multiindexed_gdf.index.get_level_values('timestep')
num_geoids = len(data_gdf['geoid'].unique())

train_shape = (num_geoids, time_window, len(features_only))

train_x_BSF_flat, train_y_BS = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year,
                                                  time_window, features_only, train_shape, pred_lag=timesteps_per_year)

valid_x_BSF_flat, valid_y_BS = make_data(multiindexed_gdf, validation_year, validation_year,
                                         time_window, features_only, train_shape, pred_lag=timesteps_per_year)

test_x_BSF_flat, test_y_BS = make_data(multiindexed_gdf, first_test_year, last_test_year,
                                       time_window, features_only, train_shape, pred_lag=timesteps_per_year)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(train_x_BSF_flat)
train_x_BSF_flat = norm_layer(train_x_BSF_flat)
valid_x_BSF_flat = norm_layer(valid_x_BSF_flat)
test_x_BSF_flat = norm_layer(test_x_BSF_flat)

top_100_idx_func = partial(top_k_idx, k=100)

In [5]:
true_deaths = multiindexed_gdf.loc[idx[:,18],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
deaths_index = true_deaths.index.values
num_tracts = len(deaths_index)
removed_tracts = 250
num_sampled = num_tracts - removed_tracts
K = 100
rng = np.random.default_rng(seed=360)

ratios = []
'''
for _ in range(5000):

    sampled_indicies = rng.choice(deaths_index, size=num_sampled, replace=False)



    top_K_orig_deaths = true_deaths.sort_values(ascending=False)[:K].sum()
    top_K_sampled_deaths = true_deaths[sampled_indicies].sort_values(ascending=False)[:K].sum()
    ratio = top_K_sampled_deaths/top_K_orig_deaths
    ratios.append(ratio)

print(f'Removed: {removed_tracts}, {np.mean(ratios):2f}')
'''

"\nfor _ in range(5000):\n\n    sampled_indicies = rng.choice(deaths_index, size=num_sampled, replace=False)\n\n\n\n    top_K_orig_deaths = true_deaths.sort_values(ascending=False)[:K].sum()\n    top_K_sampled_deaths = true_deaths[sampled_indicies].sort_values(ascending=False)[:K].sum()\n    ratio = top_K_sampled_deaths/top_K_orig_deaths\n    ratios.append(ratio)\n\nprint(f'Removed: {removed_tracts}, {np.mean(ratios):2f}')\n"

In [52]:
bprs_2019=[]
bprs_2020=[]
avg_2018 = multiindexed_gdf[(multiindexed_gdf['year']<=2018)&(multiindexed_gdf['year']>2011)].groupby(level='geoid')['deaths'].mean()
avg_2019 = multiindexed_gdf[(multiindexed_gdf['year']<=2019)&(multiindexed_gdf['year']>2012)].groupby(level='geoid')['deaths'].mean()

this_2019 = multiindexed_gdf.loc[idx[:,19],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
this_2020 = multiindexed_gdf.loc[idx[:,20],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']

rng = np.random.default_rng(seed=360)



In [45]:
%%time
for _ in range(50):
    sampled_indicies = rng.choice(deaths_index, size=num_sampled, replace=False)
    
    bprs_2019.append(fixed_top_X_new(this_2019[sampled_indicies],
                            avg_2018[sampled_indicies],
                            X=K)[-1])
    bprs_2020.append(fixed_top_X_new(this_2020[sampled_indicies],
                            avg_2019[sampled_indicies],
                            X=K)[-1])

CPU times: user 35.5 s, sys: 89.7 ms, total: 35.5 s
Wall time: 35.5 s


In [24]:
%%time
for _ in range(5):
    sampled_indicies = rng.choice(deaths_index, size=num_sampled, replace=False)
    
    for_exp_2018 = avg_2018[sampled_indicies]
    for_exp_2018.index = range(num_sampled)
    for_exp_2018=for_exp_2018.astype(np.float16)
    for_exp_2019 = this_2019[sampled_indicies]
    for_exp_2019.index = range(num_sampled)
    for_exp_2019=for_exp_2019.astype(np.float16)
    for_exp_2020 = this_2020[sampled_indicies]
    for_exp_2020.index = range(num_sampled)
    for_exp_2020=for_exp_2020.astype(np.float16)
    
    fixed_top_X_new(for_exp_2019,
                            for_exp_2018,
                            X=K)[-1]
    fixed_top_X_new(for_exp_2020,
                            for_exp_2019,
                            X=K)[-1]

CPU times: user 8.72 s, sys: 17.7 ms, total: 8.74 s
Wall time: 8.73 s


In [46]:
np.percentile(bprs_2019,2.5), np.mean(bprs_2019), np.percentile(bprs_2019, 97.5)

(0.6026420132261758, 0.6267728943374163, 0.6492241308278445)

In [47]:
np.percentile(bprs_2020,2.5), np.mean(bprs_2020), np.percentile(bprs_2020, 97.5)

(0.5690201234965635, 0.5889303224342085, 0.6099365823180255)

In [48]:
avg_bprs = (np.array(bprs_2019)+np.array(bprs_2020))/2

In [44]:
np.percentile(avg_bprs,2.5), np.mean(avg_bprs), np.percentile(avg_bprs, 97.5)

(0.590550600190544, 0.609578615927519, 0.6257661891590341)

In [49]:
np.percentile(avg_bprs,2.5), np.mean(avg_bprs), np.percentile(avg_bprs, 97.5)

(0.5893904991648975, 0.6078516083858125, 0.6230947102857879)

In [18]:
fixed_top_X(this_2019,
                            avg_2018,
                            X=K)[-1]

0.5444230769230769

In [19]:
fixed_top_X(this_2020,
                            avg_2019,
                            X=K)[-1]

0.5187334669338677

In [20]:
np.mean([0.5444230769230769,0.5187334669338677 ])

0.5315782719284723

In [23]:
this_2020.sort_values(ascending=False)[:100].sum()

499.0

In [7]:
def fixed_top_X_new(true_qtr_val, pred_qtr_val, X=10):
    top_X_predicted = pred_qtr_val.sort_values(ascending=False).iloc[:X]
    top_X_true = true_qtr_val.sort_values(ascending=False).iloc[:X]

    undisputed_top_predicted = top_X_predicted[top_X_predicted > top_X_predicted.min()]
    num_tied_spots = X - len(undisputed_top_predicted)
    undisputed_top_true = top_X_true[top_X_true > top_X_true.min()]
    num_true_ties = X - len(undisputed_top_true)

    tied_top_predicted = pred_qtr_val[pred_qtr_val == top_X_predicted.min()]
    tied_top_true = true_qtr_val[true_qtr_val == top_X_true.min()]

    error_in_top_true_ties = np.abs(tied_top_true - pred_qtr_val[tied_top_true.index]).sort_values(ascending=True)
    error_in_top_pred_ties = np.abs(true_qtr_val[tied_top_predicted.index] - tied_top_predicted).sort_values(
        ascending=True)
    top_true_tied_geoids = error_in_top_true_ties.iloc[:num_true_ties].index
    top_pred_tied_geoids = error_in_top_pred_ties.iloc[:num_tied_spots].index

    best_possible_top_true_geoids = pd.Index.union(undisputed_top_true.index, top_true_tied_geoids)
    best_possible_top_pred_geoids = pd.Index.union(undisputed_top_predicted.index, top_pred_tied_geoids)

    # True values of GEOIDS with highest actual deaths. If ties, finds tied locations that match preds best
    best_possible_true = true_qtr_val[best_possible_top_true_geoids]
    best_possible_pred = true_qtr_val[best_possible_top_pred_geoids]
    assert (len(best_possible_true) == X)
    assert (len(best_possible_pred) == X)

    best_possible_absolute = np.abs(best_possible_true.sum() - best_possible_pred.sum())
    best_possible_ratio = np.abs(best_possible_pred).sum() / np.abs(best_possible_true).sum()

    bootstrapped_tied_indices = np.random.choice(tied_top_predicted.index, (1000, num_tied_spots))
    this_idx = undisputed_top_predicted.index.values
    bootstrapped_all_indices = [np.concatenate((this_idx,bootstrap_index)) for bootstrap_index in bootstrapped_tied_indices]
    #bootstrapped_all_indices = [pd.Index.union(undisputed_top_predicted.index,
    #                                           bootstrap_index) for bootstrap_index in bootstrapped_tied_indices]
    
    denominator =  top_X_true.sum()
    numerators = [true_qtr_val[indicies].sum() for indicies in bootstrapped_all_indices] 

    bootstrapped_absolute = np.mean([np.abs(denominator - numerator)
                                     for numerator in numerators])
    bootstrapped_ratio = np.mean([numerator / denominator
                                  for numerator in numerators])

    return best_possible_absolute, best_possible_ratio, bootstrapped_absolute, bootstrapped_ratio

In [13]:
this_2019[sampled_indicies]

geoid
25013813102    0.0
25027703300    1.0
25013813205    5.0
25009213100    1.0
25017317202    0.0
              ... 
25021401100    0.0
25027761401    0.0
25001014900    0.0
25027703200    3.0
25017341600    1.0
Name: deaths, Length: 1370, dtype: float64

In [14]:
avg_2018[sampled_indicies]

geoid
25013813102    1.0
25027703300    3.0
25013813205    2.0
25009213100    0.0
25017317202    0.0
              ... 
25021401100    0.0
25027761401    0.0
25001014900    0.0
25027703200    1.0
25017341600    7.0
Name: deaths, Length: 1370, dtype: float64

In [17]:
avg_2018[sampled_indicies].index = range(len(sampled_indicies))

In [22]:
for_exp_2020.astype(np.float16)

0       1.0
1       0.0
2       0.0
3       1.0
4       1.0
       ... 
1365    0.0
1366    0.0
1367    0.0
1368    4.0
1369    0.0
Name: deaths, Length: 1370, dtype: float16

In [25]:
for_exp_2019

0       0.0
1       0.0
2       1.0
3       0.0
4       1.0
       ... 
1365    1.0
1366    1.0
1367    2.0
1368    1.0
1369    0.0
Name: deaths, Length: 1370, dtype: float16

In [26]:

length = 1620

# Generate random data
data1 = np.random.rand(length)
data2 = np.random.rand(length)

# Generate random index
index = np.random.randint(100000, 999999, length)

# Create pandas Series with random index
series1 = pd.Series(data1, index=index)
series2 = pd.Series(data2, index=index)


In [27]:
import cProfile
# Create a profiler object
profiler = cProfile.Profile()

# Start profiling
profiler.enable()

# Call your function
for _ in range(50):
    fixed_top_X_new(series1, series2, X=100)

# Stop profiling
profiler.disable()

# Print the profiling results
profiler.print_stats()

         60221221 function calls (59364421 primitive calls) in 26.309 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 3033256102.py:13(<module>)
        1    0.052    0.052   26.308   26.308 3033256102.py:9(<module>)
       50    0.006    0.000   26.256    0.525 815594956.py:1(fixed_top_X_new)
       50    0.084    0.002    8.100    0.162 815594956.py:32(<listcomp>)
       50    0.132    0.003   17.810    0.356 815594956.py:36(<listcomp>)
       50    0.041    0.001    0.041    0.001 815594956.py:38(<listcomp>)
       50    0.003    0.000    0.003    0.000 815594956.py:40(<listcomp>)
      100    0.000    0.000    0.002    0.000 <__array_function__ internals>:177(array_equal)
    50300    0.038    0.000    0.162    0.000 <__array_function__ internals>:177(concatenate)
      100    0.000    0.000    0.006    0.000 <__array_function__ internals>:177(mean)
      200    0.000    0.0

In [28]:
%%time
for _ in range(50):
    fixed_top_X_new(series1, series2, X=100)

CPU times: user 12.8 s, sys: 48.8 ms, total: 12.8 s
Wall time: 12.8 s


In [51]:
%%time
for _ in range(5):
    fixed_top_X(for_exp_2020, for_exp_2019, X=100)

CPU times: user 6.28 s, sys: 29.2 ms, total: 6.31 s
Wall time: 6.28 s


In [31]:
profiler = cProfile.Profile()

# Start profiling
profiler.enable()
X=100
pred_qtr_val = this_2019
true_qtr_val = this_2020

top_X_predicted = pred_qtr_val.sort_values(ascending=False).iloc[:X]
top_X_true = true_qtr_val.sort_values(ascending=False).iloc[:X]

undisputed_top_predicted = top_X_predicted[top_X_predicted > top_X_predicted.min()]
num_tied_spots = X - len(undisputed_top_predicted)
undisputed_top_true = top_X_true[top_X_true > top_X_true.min()]
num_true_ties = X - len(undisputed_top_true)

tied_top_predicted = pred_qtr_val[pred_qtr_val == top_X_predicted.min()]
tied_top_true = true_qtr_val[true_qtr_val == top_X_true.min()]

error_in_top_true_ties = np.abs(tied_top_true - pred_qtr_val[tied_top_true.index]).sort_values(ascending=True)
error_in_top_pred_ties = np.abs(true_qtr_val[tied_top_predicted.index] - tied_top_predicted).sort_values(
    ascending=True)
top_true_tied_geoids = error_in_top_true_ties.iloc[:num_true_ties].index
top_pred_tied_geoids = error_in_top_pred_ties.iloc[:num_tied_spots].index

best_possible_top_true_geoids = pd.Index.union(undisputed_top_true.index, top_true_tied_geoids)
best_possible_top_pred_geoids = pd.Index.union(undisputed_top_predicted.index, top_pred_tied_geoids)

# True values of GEOIDS with highest actual deaths. If ties, finds tied locations that match preds best
best_possible_true = true_qtr_val[best_possible_top_true_geoids]
best_possible_pred = true_qtr_val[best_possible_top_pred_geoids]
assert (len(best_possible_true) == X)
assert (len(best_possible_pred) == X)

best_possible_absolute = np.abs(best_possible_true.sum() - best_possible_pred.sum())
best_possible_ratio = np.abs(best_possible_pred).sum() / np.abs(best_possible_true).sum()

bootstrapped_tied_indices = np.random.choice(tied_top_predicted.index, (1000, num_tied_spots))
bootstrapped_all_indices = [pd.Index.union(undisputed_top_predicted.index,
                                           bootstrap_index) for bootstrap_index in bootstrapped_tied_indices]

denominator =  top_X_true.sum()
numerators = [true_qtr_val[indicies].sum() for indicies in bootstrapped_all_indices] 

bootstrapped_absolute = np.mean([np.abs(denominator - numerator)
                                 for numerator in numerators])
bootstrapped_ratio = np.mean([numerator / denominator
                              for numerator in numerators])

# Stop profiling
profiler.disable()

# Print the profiling results
profiler.print_stats()


         4452780 function calls (4379630 primitive calls) in 2.849 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 2905831795.py:10(<module>)
        1    0.000    0.000    0.001    0.001 2905831795.py:12(<module>)
        1    0.000    0.000    0.000    0.000 2905831795.py:13(<module>)
        1    0.000    0.000    0.001    0.001 2905831795.py:14(<module>)
        1    0.000    0.000    0.000    0.000 2905831795.py:15(<module>)
        1    0.000    0.000    0.001    0.001 2905831795.py:17(<module>)
        1    0.000    0.000    0.001    0.001 2905831795.py:18(<module>)
        1    0.000    0.000    0.001    0.001 2905831795.py:20(<module>)
        1    0.000    0.000    0.001    0.001 2905831795.py:21(<module>)
        1    0.000    0.000    0.000    0.000 2905831795.py:23(<module>)
        1    0.000    0.000    0.000    0.000 2905831795.py:24(<module>)
        1    0.000 

In [33]:
undisputed_top_predicted.index

Index(['25017310200', '25005651400', '25023545200', '25005631400',
       '25009251600', '25005652700', '25009206700', '25023510200',
       '25027731400', '25009205900', '25009206000', '25009206200',
       '25023500104', '25009251300', '25013810700', '25009206800',
       '25003900100', '25003900200', '25005641200', '25021418002',
       '25013802101', '25013802300', '25017310300', '25003935300',
       '25009210100', '25009251800', '25027732700', '25027732400',
       '25013813205', '25027754200', '25025081900', '25027753200',
       '25023542101', '25005642000', '25005640500', '25027731203',
       '25027737100', '25005655100', '25025170800', '25005641101'],
      dtype='object', name='geoid')

In [48]:
%%time 
this_idx = undisputed_top_predicted.index.values
bootstrapped_all_indices = [np.concatenate((this_idx,bootstrap_index)) for bootstrap_index in bootstrapped_tied_indices]

CPU times: user 2.22 ms, sys: 5 µs, total: 2.22 ms
Wall time: 2.23 ms


In [38]:
%%time 
bootstrapped_all_indices = [pd.concat((undisputed_top_predicted.index ,bootstrap_index)) for bootstrap_index in bootstrapped_tied_indices]

TypeError: cannot concatenate object of type '<class 'pandas.core.indexes.base.Index'>'; only Series and DataFrame objs are valid

In [43]:
bootstrapped_tied_indices

array([['25005651700', '25027726200', '25027731002', ..., '25005651001',
        '25013800400', '25027726200'],
       ['25025160501', '25025040200', '25013802601', ..., '25009260800',
        '25005613600', '25009221400'],
       ['25027733000', '25005651001', '25005653101', ..., '25005640202',
        '25003921300', '25013800101'],
       ...,
       ['25003921300', '25005651100', '25009250600', ..., '25025160501',
        '25009260800', '25025160501'],
       ['25013812002', '25005652000', '25025060800', ..., '25013800900',
        '25009221400', '25001011600'],
       ['25023510900', '25005653101', '25023545400', ..., '25021417801',
        '25017314301', '25009250600']], dtype=object)

In [46]:
bootstrapped_tied_indices[0]

array(['25005651700', '25027726200', '25027731002', '25021417801',
       '25025060800', '25005651001', '25005652600', '25005613600',
       '25025090100', '25027732600', '25027733000', '25013811301',
       '25005613800', '25023522101', '25005613600', '25009221500',
       '25013800101', '25009251400', '25017341200', '25013812002',
       '25005651100', '25023522101', '25025110201', '25013800400',
       '25025170502', '25023522101', '25005652000', '25009261000',
       '25009250600', '25027726200', '25027733000', '25023542302',
       '25025110201', '25009261000', '25005651700', '25027733000',
       '25027726200', '25023545400', '25027726200', '25027732600',
       '25021417801', '25003900400', '25005652000', '25025170702',
       '25027731900', '25017314301', '25009221500', '25009221500',
       '25005653101', '25025160501', '25009251400', '25025170702',
       '25013812002', '25009250600', '25027726200', '25005652600',
       '25025060800', '25005651001', '25013800400', '250277262

In [69]:
fixed_top_X_new(this_2020, this_2019, X=100)

(238.0, 0.5230460921843687, 240.463, 0.5181102204408817)

In [70]:
fixed_top_X(this_2020, this_2019, X=100)

(238.0, 0.5230460921843687, 240.493, 0.5180501002004009)